In [3]:
#Import Required Python Packages
import folium
import pandas as pd

In [4]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [7]:
#importing the datasets
spacex_df=pd.read_csv("../data/spacex_launch_geo.csv")
print("datasets available")

datasets available


#  Plot Launch Sites on the Map and Analyze Locations

In [8]:
# Select relevant sub-columns from the dataframe
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]

# Group by Launch Site and get the first entry for each site
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()

# Keep only Launch Site name and coordinates
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


<b>Initialize Map Centered at NASA Johnson Space Center<b/>

In [9]:
# Start location is NASA Johnson Space Center with given coordinates
nasa_coordinate = [29.559684888503615, -95.0830971930759]

# Initialize the folium map centered at NASA Johnson Space Center
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)


In [10]:
# Create an orange circle with a popup label
circle = folium.Circle(
    nasa_coordinate,
    radius=1000,
    color='#d35400',
    fill=True
).add_child(folium.Popup('NASA Johnson Space Center'))

# Create a marker with a custom text icon label
marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
    )
)

# Add circle and marker to the map
site_map.add_child(circle)
site_map.add_child(marker)

In [13]:
from folium.plugins import MarkerCluster

# Create a MarkerCluster object to manage marker grouping
marker_cluster = MarkerCluster()

# Create a new column 'marker_color' in spacex_df based on 'class' column:
# Green marker for success (class=1), Red marker for failure (class=0)
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Loop through each launch record in spacex_df
for index, record in spacex_df.iterrows():
    # Create a folium Marker for the launch site's coordinates
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        # Corrected marker icon color based on launch success/failure
        icon=folium.Icon(color=record['marker_color']),
        # Add popup text to show launch site and class
        popup=f"Launch Site: {record['Launch Site']}<br>Class: {record['class']}"
    )
    # Add the marker to the marker cluster
    marker_cluster.add_child(marker)

# Add the cluster of markers to the site map
site_map.add_child(marker_cluster)

# Display the updated map
site_map


# Calculate Distance Between Launch Site and Closest Coastline


In [14]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# find coordinate of the closet coastline

In [16]:
# Define launch site coordinate (example: NASA Johnson Space Center)
launch_site_lat = 29.559684888503615
launch_site_lon = -95.0830971930759

# Define closest coastline coordinate (example coordinate near Cape Canaveral)
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate distance between launch site and coastline
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print(f"Distance between launch site and coastline: {distance_coastline:.2f} km")


Distance between launch site and coastline: 1414.31 km


In [19]:
# Example closest coastline coordinates (replace with actual values)
coordinate = [28.56367, -80.57163]  # Lat, Long of closest coastline point

# Example distance value (you should calculate this beforehand)
distance = 5.0  # example distance in kilometers

# Now you can create the marker
distance_marker = folium.Marker(
    coordinate,  # Closest coastline point coordinates
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
    )
)

# Add marker to map
site_map.add_child(distance_marker)


#  Are launch sites in close proximity to railways?

In [21]:
# Define launch site coordinate (latitude, longitude)
launch_site_coord = [28.5623, -80.5774]  # Replace with actual launch site coords

# Example railway coordinate (replace with actual data)
railway_coord = [28.5, -80.6]

# Calculate distance from launch site to railway
distance_to_railway = calculate_distance(launch_site_coord[0], launch_site_coord[1], railway_coord[0], railway_coord[1])

print(f"Distance to nearest railway: {distance_to_railway:.2f} KM")


Distance to nearest railway: 7.27 KM


# Are launch sites in close proximity to highways?

In [23]:
# Assuming you have highway coordinates, calculate distance to launch site
# Example highway coordinate (replace with actual data)
highway_coord = [28.6, -80.7]

# Calculate distance from launch site to highway
distance_to_highway = calculate_distance(launch_site_coord[0], launch_site_coord[1], highway_coord[0], highway_coord[1])

print(f"Distance to nearest highway: {distance_to_highway:.2f} KM")

# Conclusion based on threshold (e.g., 10 KM)
if distance_to_highway <= 10:
    print("Launch site is in close proximity to a highway.")
else:
    print("Launch site is NOT in close proximity to a highway.")


Distance to nearest highway: 12.69 KM
Launch site is NOT in close proximity to a highway.


#  Are launch sites in close proximity to coastline?

In [24]:
# distance_to_coastline calculated previously

print(f"Distance to nearest coastline: {distance:.2f} KM")

if distance <= 10:
    print("Launch site is in close proximity to the coastline.")
else:
    print("Launch site is NOT in close proximity to the coastline.")


Distance to nearest coastline: 5.00 KM
Launch site is in close proximity to the coastline.


#  Do launch sites keep certain distance away from cities?

In [25]:
# Assuming city coordinates are known
city_coord = [29.7, -95.3]

# Calculate distance from launch site to city
distance_to_city = calculate_distance(launch_site_coord[0], launch_site_coord[1], city_coord[0], city_coord[1])

print(f"Distance to nearest city: {distance_to_city:.2f} KM")

# Threshold example: 20 KM
if distance_to_city >= 20:
    print("Launch site keeps a reasonable distance away from the city.")
else:
    print("Launch site is close to a city.")


Distance to nearest city: 1435.06 KM
Launch site keeps a reasonable distance away from the city.
